<a href="https://colab.research.google.com/github/minnji88/DeepLearning2020/blob/master/3_5_%EB%89%B4%EC%8A%A4%EA%B8%B0%EC%82%AC%EB%B6%84%EB%A5%98_%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 로이터 데이터셋

In [ ]:
from keras.datasets import reuters
# 가장 자주 등장하는 단어 10,000개로 제한
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words = 10000)

In [ ]:
len(train_data)

In [ ]:
len(test_data)

In [ ]:
print(train_data[10])

## 데이터 준비

*레이블을 벡터로 바꾸는 방법*

1. 레이블의 리스트를 정수 텐서로 변환
2. 원-핫 인코딩 사용

In [ ]:
# 코드 3-14 데이터 인코딩하기
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data) # 훈련 데이터 벡터 변환
x_test = vectorize_sequences(test_data) # 테스트 데이터 벡터 변환

####  원-핫 인코딩

In [ ]:
# 레이블 벡터 변환
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

### 모델 구성
출력결과가 2개로 나누어지는 것이 아닌 46개라는 차이

16차원은 46개의 데이터를 구분하지 못할 수 있으니 이번에는 64개의 유닛을 이용합니다.



마지막 출력이 46입니다. 각 입력 샘플에 대해 46차원 벡터를 출력한다는 뜻입니다.
softmax 활성화 함수를 사용하였습니다. 이는 46개의 출력 클래스에 대한 확률 분포를 출력하며, 총합은 1입니다.
이런 분류에서 손실 함수는 categorical_crossentropy를 사용합니다. 이 함수는 두 확률 분포 사이의 거리를 측정하고, 확률 분포 사의 거리를 최소화하며 최적화 합니다.


In [ ]:
# 코드 3-15 모델 정의하기
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

In [ ]:
# 모델 컴파일하기
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## 훈련 검증

In [ ]:
# 검증세트 준비하기
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

In [ ]:
# 코드 3-18 모델 훈련하기
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

In [ ]:
# 훈련과 검증 손실 그리기
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()
print(epochs)

In [ ]:
# 훈련과 검증 정확도
plt.clf() #그래프를 초기화합니다.

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# 아홉번째 에포크 이후 과대적합 발생
# 코드 3-21 모델을 처음부터 다시 훈련하기
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=9,
          batch_size=512,
          validation_data=(x_val, y_val))

results = model.evaluate(x_test, one_hot_test_labels)


In [ ]:
results     # 78% 정확도 달성

### 새로운 데이터에 대해 예측하기

In [ ]:
predictions = model.predict(x_test)

In [ ]:
predictions[0].shape

In [ ]:
np.sum(predictions[0])

In [ ]:
np.argmax(predictions[0])